<a href="https://colab.research.google.com/github/EduardoFCarvalho/personal_workinstructor/blob/main/Personal_Work_Instructior_Desafio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-genai

In [ ]:
# Framework ADK
!pip install -q google-adk

In [ ]:
# Configuração API Key do Google Gemini
import os
from google.colab import userdata

# Configuração o cliente da SDK do Gemini
from google import genai

# Configuração tipos para o chat
from google.genai import types

In [ ]:
# estrutura para os agentes
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Definições de ambiente

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Configuração do chat

chat_config = types.GenerateContentConfig(
    system_instruction = """
    Você é um assistente pessoal que vai ajudar o usuário que está para iniciar em uma nova vaga de emprego.
    Você será educado, com palavras positivas enquanto obtém as principais informações sobre o perfil do
    usuário e a vaga de emprego dele.
    """
)

chat = client.chats.create(model=MODEL_ID, config=chat_config)

In [ ]:
# Variavel resposavel por finalizar a conversa
# Encontrei conflito com o exit no colab e foi necessário aplicar de outra forma
app_should_exit = False

def get_input_or_exit(prompt_message):

    global app_should_exit
    user_input = input(prompt_message)
    if user_input.lower() == 'exit':
        print("\nObrigado, espero que possamos conversar novamente em breve. Até logo!")
        app_should_exit = True
        return
    return user_input

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
####################################################
# --- Agente 1: Identificador de perfil --- #
####################################################
def agente_identificador(topico, data_de_hoje):
  identificador = Agent(
      name = "agente_identificador",
      model = "gemini-2.0-flash",
      instruction = """
      Você é um assistente de identificação de perfil de usuário. Responsável por definir a persona
      com base na idade e vaga de trabalho do usuário.
      """,
      description = "Agente de identificação de perfil de usuário"
  )
  entrada_do_agente_identificador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  perfil_usuario = call_agent(identificador, entrada_do_agente_identificador)

  return perfil_usuario

In [ ]:
###########################################
# --- Agente 2: Interpretador da vaga --- #
###########################################
def agente_interpretador(topico, data_de_hoje):
  interpretador = Agent(
      name = "agente_interpretador",
      model = "gemini-2.0-flash",
      instruction = """
      Você é um assistente de identificação de vagas de trabalho. Sua tarefa será receber as informações
      da vaga que o usuário irá atuar, separar os tópicos e contextos e com base nisso usar a busca do
      Google (google_search) para obter o máximo de informações atuais possíveis sobre todos os temas.
      Importante que para cada tópico trazer de forma sucinta os conteúdos para não ficar extenso, no
      máximo 3 linhas por tópico.
      """,
      description = "Agente de identificação de perfil de usuário",
      tools = [google_search]
  )
  entrada_do_agente_interpretador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  perfil_da_vaga = call_agent(interpretador, entrada_do_agente_interpretador)

  return perfil_da_vaga

In [ ]:
##################################
# --- Agente 3: Estrategista --- #
##################################
def agente_estrategista(topico, data_de_hoje):
  estrategista = Agent(
      name = "agente_estrategista",
      model = "gemini-2.0-flash",
      instruction = """
      Você é um assistente de estratégias. Você vai trabalhar com o perfil do usuário, prazo para iniciar
      na nova função, conhecimentos que o usuário tem sobre o assunto e informações sobre a função que ele
      vai exercer.
      Com base nisso você vai planejar um cronograma de estudos para que a pessoa esteja mais preparada
      para os desafios que encontrará. Busque também no Google (google_search) sobre cursos que tenham
      relevância e disponibilize os links. No máximo 6 links.
      """,
      description = "Agente de identificação de perfil de usuário",
      tools = [google_search]
  )
  entrada_do_agente_estrategista = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

  perfil_da_vaga = call_agent(estrategista, entrada_do_agente_estrategista)

  return perfil_da_vaga

In [ ]:
##################################
# --- Agente 0: Orquestrador --- #
##################################
def agente_orquestrador(age: str, job: str, time_ti_begin: str, know_things: str, informations: str, data_de_hoje: str) -> str:
    print("\nChamando agente_orquestrador...")

    # 1. Prepara e chama o agente_identificador
    perfil_input_string = f"Idade: {age}, Cargo: {job}, Conhecimento prévio: {know_things}"
    print(" -> Chamando agente_identificador para processar o perfil...")
    perfil_identificado = agente_identificador(perfil_input_string, data_de_hoje)
    display(to_markdown(perfil_identificado))
    print(" <- Resposta do agente_identificador recebida.")

    # 2. Prepara e chama o agente_interpretador (Este será o último agente no fluxo atual)
    descricao_vaga_input = informations
    print(" -> Chamando agente_interpretador para processar a vaga...")
    vaga_interpretada = agente_interpretador(descricao_vaga_input, data_de_hoje)
    display(to_markdown(vaga_interpretada))
    print(" <- Resposta do agente_interpretador recebida.")

    # 3. Criação da estratégia de estudos
    infos_para_estrategia = f"""Perfil do usuário: {perfil_identificado}, Prazo de inicio: {time_ti_begin},
    Descrição da vaga: {vaga_interpretada}"""
    print(" -> Chamando agente_estrategista para processar a estratégia...")
    estrategia_definida = agente_estrategista(infos_para_estrategia, data_de_hoje)
    # display(to_markdown(estrategia_definida))
    # print(" <- Resposta do agente_estrategista recebida.")

    # 4. Retorna o resultado do último agente (agente_interpretador)
    print("Orquestração concluída.")
    return estrategia_definida

In [ ]:
# Lógica principal do chat - Atualizada para chamar o orquestrador
def main_chat_logic():
    global app_should_exit

    print("Olá, sou seu assistente pessoal e estou aqui para ajudo-lo com seu novo cargo!\n")
    print("Para iniciarmos vou precisar de algumas informações.\n")
    print("Se em qualquer momento você quiser finalizar nossa conversa basta digitar exit. Vamos começar?\n\n")

    # Coleta as informações do usuário
    age = get_input_or_exit("Qual sua idade? ")
    if app_should_exit:
        return

    job = get_input_or_exit("Qual cargo você vai atuar? ")
    if app_should_exit:
        return

    time_to_begin = get_input_or_exit("Em quanto tempo você vai iniciar as atividades? ")
    if app_should_exit:
        return

    know_things = get_input_or_exit("Descreva seus conhecimentos atuais com relação as informações da vaga: ")
    if app_should_exit:
        return

    informations = get_input_or_exit("Descreva a vaga que você vai atuar: ")
    if app_should_exit:
        return

    print(f"\nÓtimo! Recebi suas informações: Idade - {age}, Cargo - {job}.")
    print("Vou processar os detalhes da vaga que você me passou...")

    # Obtém a data atual para passar aos agentes
    data_de_hoje = date.today().strftime("%Y-%m-%d")

    # Chama a função orquestradora para processar as informações
    final_response_from_orchestrator = agente_orquestrador(age, job, time_to_begin, know_things, informations, data_de_hoje)

    # Exibe a resposta final
    print("\nProcessamento concluído! Aqui estão as informações sobre a vaga:\n")
    display(to_markdown(final_response_from_orchestrator))
    print("\nEspero ter ajudado! Bons estudos e boa sorte em seu novo emprego!")


# Inicia a aplicação
if __name__ == "__main__":
    main_chat_logic()
    if app_should_exit:
        print("Aplicação encerrada pelo usuário.")
    else:
        print("\nAtividades concluídas. Até mais!")
